### Едноставен систем за препорака на ентитети

Со користење на Spark можеме да направиме едноставен систем за препорака на филмови за одреден корисник базирано врз неговите веќе поставени рејтинзи за филмови. За ова ќе ја искористиме библиотеката MLLib која ја нуди Спарк.

Можеме многу едноставно да го направиме ова со следнава Python скрипта.

1. Ќе креираме Спарк сесија
2. Ќе ги прочитаме податоците во вид на dictionary
3. Ќе ги претвориме во RDD со (userID, movieID, rating)
4. Ќе креираме DataFrame и ќе го ставиме во cache
5. Ќе креираме ALS колаборативен модел за филтрирање од целосниот податочен сет
- ова е ML концепт кој ќе служи за тренирање на нашиот алгоритам 

Со стартување дел по дел на следниве кодови можеме и да видиме како всушност работи сето ова:


Го инсталираме потребните библиотеки

In [5]:
! pip install pyspark py4j

Ги наведуваме сите библиотеки кои ќе ги користиме и дополнително дефинираме функции за читање и парсирање на имињата на филмовите

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import lit

def loadMovieNames():
  movieNames = {}
  with open("u.item",  encoding = "ISO-8859-1") as f:
    for line in f:
      fields = line.split('|')
      movieNames[int(fields[0])] = fields[1]
  return movieNames

def parseInput(line):
  fields = line.value.split()
  return Row(userID = int(fields[0]), movieID=int(fields[1]), rating=float(fields[2]))

Креираме Spark сесија која ја сместуваме во променливата spark

In [6]:
spark = SparkSession.builder.appName("MovieRecs").getOrCreate()

Ќе ги претвориме во RDD со (userID, movieID, rating)

In [7]:
movieNames = loadMovieNames()
lines = spark.read.text("u.data").rdd

Ќе креираме DataFrame и ќе го ставиме во cache

In [8]:
ratingsRDD = lines.map(parseInput)
ratings = spark.createDataFrame(ratingsRDD).cache()
als = ALS(maxIter=5, regParam=0.01, userCol="userID", itemCol="movieID", ratingCol="rating")
model = als.fit(ratings)

Ќе креираме ALS колаборативен модел за филтрирање од целосниот податочен сет. Ги прикажуваме топ 20-те филмови препорачани за корисник со ID=0

In [9]:
print("\nRatings for user ID 0:")
userRatings = ratings.filter("userID = 0")
for rating in userRatings.collect():
  print(movieNames[rating['movieId']], rating['rating'])

print("\nTop 20 recommendations:")
ratingCounts = ratings.groupBy("movieID").count().filter("count > 100")
popularMovies = ratingCounts.select("movieID").withColumn('userID', lit(0))
recommendations = model.transform(popularMovies)
topRecommendations = recommendations.sort(recommendations.prediction.desc()).take(20)

for recommendation in topRecommendations:
  print(movieNames[recommendation['movieID']], recommendation['prediction'])

spark.stop()


Ratings for user ID 0:

Top 20 recommendations:
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963) nan
Batman Forever (1995) nan
What's Eating Gilbert Grape (1993) nan
Amadeus (1984) nan
Cinderella (1950) nan
Star Trek: First Contact (1996) nan
Donnie Brasco (1997) nan
Gattaca (1997) nan
Clueless (1995) nan
Singin' in the Rain (1952) nan
Jungle2Jungle (1997) nan
Outbreak (1995) nan
Down Periscope (1996) nan
True Lies (1994) nan
Last of the Mohicans, The (1992) nan
Jerry Maguire (1996) nan
Wag the Dog (1997) nan
Beauty and the Beast (1991) nan
Nikita (La Femme Nikita) (1990) nan
Braveheart (1995) nan
